In [1]:
########################start##############################
import pandas as pd
df=pd.read_csv('./datas/Data_new.csv',index_col=0)
df.index=pd.to_datetime(df.index,format='%Y-%m-%d') 
df_train=df.loc['2020-01-07':'2021-04-02'] 
df_test=df.loc['2021-04-02':'2021-04-23']

name=['cityin','cityto','incity','city']
name1=['cityin','cityto','incity']
x_train=df_train.drop(name,axis=1).values  
y_train=df_train[name1].values
x_test=df_test.drop(name,axis=1).values  
y_test=df_test[name1].values
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

#####################标准化##############################
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(x_train)
x_train = ss.transform(x_train) 
x_test = ss.transform(x_test)

(57504, 43) (19133, 43) (57504, 3) (19133, 3)


In [2]:
import csv
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import joblib
plt.rcParams['font.sans-serif'] = ['SimHei']  # 中文字体设置-黑体
plt.rcParams['axes.unicode_minus'] = False  # 解决保存图像是负号'-'显示为方块的问题
sns.set(font='SimHei',font_scale=1.5)  # 解决Seaborn中文显示问题并调整字体大小
sns.set_style("white")
def mkdir(path):
    folder = os.path.exists(path)
    if not folder:                   
        os.makedirs(path)            
        print("---  new folder...  ---")
        print("---  OK  ---")
    else:
        print("---  There is this folder!  ---")
mkdir("./results/")        
from sklearn.metrics import mean_absolute_error,mean_squared_error,median_absolute_error,r2_score,mean_squared_log_error

def calculate(y_true, y_predict, n, p):
    y_true = y_true
    y_predict = y_predict
    mse = mean_squared_error(y_true, y_predict)
    rmse = np.sqrt(mean_squared_error(y_true, y_predict))
    mae = mean_absolute_error(y_true, y_predict)
    r2 = r2_score(y_true, y_predict)
    mad = median_absolute_error(y_true, y_predict)
    mape = np.mean(np.abs((y_true - y_predict) / y_true)) * 100
    r2_adjusted = 1-((1-r2)*(n-1))/(n-p-1)
    # print('MSE: ', mse)
    # print('RMSE: ', rmse)
    # print('MAE: ', mae)
    # print('R2: ', r2)
    # print('MAD:', mad)
    # print('MAPE:', mape)
    # print('R2_Adjusted: ',r2_adjusted)
    return mse,rmse,mae,r2,mad,mape,r2_adjusted
# 将参数和评估结果写入文件
def write_csv_result(path_1,path_2,all_metrics,all_parameter):
    with open(path_1,"a",encoding="utf-8",newline="")as f:
        f = csv.writer(f)
        f.writerow(all_metrics)
    with open(path_2,"a",encoding="utf-8",newline="")as f:
        f = csv.writer(f)
        f.writerow(all_parameter)


---  There is this folder!  ---


In [4]:
from sklearn.ensemble import GradientBoostingRegressor
import csv
import os
from sklearn.multioutput import RegressorChain

def adjust_gbdt(train_x,test_x,train_y,test_y,name):
    path_a = "./results/" + "Gbdt_" + name + "_" + "assess.csv"
    path_p = "./results/" + "Gbdt_" + name + "_" + "parameter.csv"
    path_b = "./results/" + "Gbdt_" + name + "_" + "best.csv"
    model_path = "./results/" + "Gbdt_" + name + "_" + "best.m"
    all_assessed_values = []
    all_parameter = []
    random_state = 17
    n_estimators = [100,200,300,400,500,600,700,800]
    learning_rate = [0.1,1e-2,0.2,2e-2,1e-3,0.3]
    loss = ["huber","ls","lad"]
    subsample = [1,0.6,0.2]
    min_samples_split = [2]
    max_depth = [3,7,11]
    min_samples_leaf = [1]
    all_nb = len(max_depth) * len(n_estimators) * len(learning_rate) * len(loss) * len(subsample) * len(min_samples_leaf) * len(min_samples_split)
    num = 1
    # 用于重启训练模型，提高效率，不重复跑相同的实验
    if(os.path.exists(path_a)):
        data = pd.read_csv(path_a)
        if(data.shape[0]>1):
            nums = int(data.values[-1,0])
        else:
            nums = 0
    else:
        nums = 0
        col_a = ['num','mse','rmse','mae','r2','mad','mape','r2_adjusted']
        col_p = ['num','n_estimators','learning_rate','loss','subsample','min_samples_split','max_depth','min_samples_leaf']
        write_csv_result(path_a,path_p,col_a,col_p)
    # 用于保存最好的模型
    if(os.path.exists(path_b)):
        best_results = pd.read_csv(path_b)
        if(best_results.shape[0]>1):
            best_result = best_results["rmse"].values[-1]
        else:
            best_result = 10*10**30
    else:
        with open(path_b,"a",encoding="utf-8",newline="")as f:
            f = csv.writer(f)
            f.writerow(['num','mse','rmse','mae','r2','mad','mape','r2_adjusted'])
        best_result = 10*10**30
    for n in n_estimators:
        for l in learning_rate:
            for lo in loss:
                for sub in subsample:
                    for mi in min_samples_split:
                        for ma in max_depth:
                            for ms in min_samples_leaf:
                                if(nums>=num):
                                    num = num+1
                                else:
                                    print("train...{}/{}".format(num,all_nb))
                                    model = GradientBoostingRegressor(random_state=random_state,n_estimators=n,learning_rate=l,loss=lo,subsample=sub,max_depth=ma,min_samples_split=mi,min_samples_leaf=ms)
                                    wrapper = RegressorChain(model)
                                    wrapper.fit(train_x,train_y)
                                    pred_test = wrapper.predict(test_x)
            #     
                                    y_test=test_y       
                                    y_test1=[]
                                    for i in y_test:
                                        y_test1.append(i[0])
                                    pred_test1=[]
                                    for i in pred_test:
                                        pred_test1.append(i[0])
                                    y_test2=[]
                                    for i in y_test:
                                        y_test2.append(i[1])
                                    pred_test2=[]
                                    for i in pred_test:
                                        pred_test2.append(i[1])
                                    y_test3=[]
                                    for i in y_test:
                                        y_test3.append(i[2])
                                    pred_test3=[]
                                    for i in pred_test:
                                        pred_test3.append(i[2])
                                    pred_test1 = np.array(pred_test1).reshape(-1,1)
                                    y_test1 = np.array(y_test1).reshape(-1,1)
                                    sample_n = pred_test1.shape[0]
                                    feature_n = x_test.shape[1]
                                    mse1,rmse1,mae1,r21,mad1,mape1,r2_adjusted1 = calculate(y_test1,pred_test1,sample_n,feature_n)
                                

                                    pred_test2 = np.array(pred_test2).reshape(-1,1)
                                    y_test2 = np.array(y_test2).reshape(-1,1)
                                    sample_n = pred_test2.shape[0]
                                    feature_n = x_test.shape[1]
                                    mse2,rmse2,mae2,r22,mad2,mape2,r2_adjusted2 = calculate(y_test2,pred_test2,sample_n,feature_n)
                                    

                                    pred_test3 = np.array(pred_test3).reshape(-1,1)
                                    y_test3 = np.array(y_test3).reshape(-1,1)
                                    sample_n = pred_test3.shape[0]
                                    feature_n = x_test.shape[1]
                                    mse3,rmse3,mae3,r23,mad3,mape3,r2_adjusted3 = calculate(y_test3,pred_test3,sample_n,feature_n)
                    

                                    pred_test4 = pred_test.reshape(-1,1)
                                    y_test4 = y_test.reshape(-1,1)
                                    sample_n = pred_test4.shape[0]
                                    feature_n = x_test.shape[1]
                                    mse,rmse,mae,r2,mad,mape,r2_adjusted = (mse1+mse2+mse3)/3,(rmse1+rmse2+rmse3)/3,(mae1+mae2+mae3)/3,(r21+r22+r23)/3,(mad1+mad2+mad3)/3,(mape1+mape2+mape3)/3,(r2_adjusted1+r2_adjusted2+r2_adjusted3)/3
                                    all_m = [num,mse,rmse,mae,r2,mad,mape,r2_adjusted]
                                    all_p = [num,n,l,lo,sub,mi,ma,ms]
                                    # print(all_m)
                                    write_csv_result(path_a,path_p,all_m,all_p)              
                                    if(rmse < best_result):
                                        joblib.dump(filename=model_path,value=model,compress=True)
                                        # joblib.dump(model,model_path)
                                        with open(path_b,"w",encoding="utf-8",newline="")as f:
                                            f = csv.writer(f)
                                            f.writerow([num,mse,rmse,mae,r2,mad,mape,r2_adjusted])
                                            f.writerow([num,mse1,rmse1,mae1,r21,mad1,mape1,r2_adjusted1])
                                            f.writerow([num,mse2,rmse2,mae2,r22,mad2,mape2,r2_adjusted2])
                                            f.writerow([num,mse3,rmse3,mae3,r23,mad3,mape3,r2_adjusted3])
                                        best_result = rmse
                                        print("current best result, mse:{},mae:{},rmse:{},mad:{},r2:{},mape:{},r2 adjusted:{}".format(mse,mad,rmse,mad,r2,mape,r2_adjusted))
                                    print("end....",num)
                                    num = num+1
                                    print("--------------------------------")    
                    
name='Migration'
adjust_gbdt(x_train,x_test,y_train,y_test,name)

train...1/1296
current best result, mse:0.18667763580877428,mae:0.09666571557175191,rmse:0.43199474810432187,mad:0.09666571557175191,r2:0.8195741510526592,mape:14.995033359124749,r2 adjusted:0.8191677226643342
end.... 1
--------------------------------
train...2/1296
current best result, mse:0.18215556638766683,mae:0.08333156558414372,rmse:0.42386423076334173,mad:0.08333156558414372,r2:0.8300457764704624,mape:12.599594660919754,r2 adjusted:0.8296629365306138
end.... 2
--------------------------------
train...3/1296
end.... 3
--------------------------------
train...4/1296
end.... 4
--------------------------------
train...5/1296
end.... 5
--------------------------------
train...6/1296
end.... 6
--------------------------------
train...7/1296
end.... 7
--------------------------------
train...8/1296
end.... 8
--------------------------------
train...9/1296
end.... 9
--------------------------------
train...10/1296
end.... 10
--------------------------------
train...11/1296
current best

end.... 121
--------------------------------
train...122/1296
end.... 122
--------------------------------
train...123/1296
end.... 123
--------------------------------
train...124/1296
end.... 124
--------------------------------
train...125/1296
end.... 125
--------------------------------
train...126/1296
end.... 126
--------------------------------
train...127/1296
end.... 127
--------------------------------
train...128/1296
end.... 128
--------------------------------
train...129/1296
end.... 129
--------------------------------
train...130/1296
end.... 130
--------------------------------
train...131/1296
end.... 131
--------------------------------
train...132/1296
end.... 132
--------------------------------
train...133/1296
end.... 133
--------------------------------
train...134/1296
end.... 134
--------------------------------
train...135/1296
end.... 135
--------------------------------
train...136/1296
end.... 136
--------------------------------
train...137/1296
end.... 

end.... 254
--------------------------------
train...255/1296
end.... 255
--------------------------------
train...256/1296
end.... 256
--------------------------------
train...257/1296
end.... 257
--------------------------------
train...258/1296
end.... 258
--------------------------------
train...259/1296
end.... 259
--------------------------------
train...260/1296
end.... 260
--------------------------------
train...261/1296
end.... 261
--------------------------------
train...262/1296
end.... 262
--------------------------------
train...263/1296
end.... 263
--------------------------------
train...264/1296
end.... 264
--------------------------------
train...265/1296
end.... 265
--------------------------------
train...266/1296
end.... 266
--------------------------------
train...267/1296
end.... 267
--------------------------------
train...268/1296
end.... 268
--------------------------------
train...269/1296
end.... 269
--------------------------------
train...270/1296
end.... 

end.... 384
--------------------------------
train...385/1296
end.... 385
--------------------------------
train...386/1296
end.... 386
--------------------------------
train...387/1296
end.... 387
--------------------------------
train...388/1296
end.... 388
--------------------------------
train...389/1296
end.... 389
--------------------------------
train...390/1296
end.... 390
--------------------------------
train...391/1296
end.... 391
--------------------------------
train...392/1296
end.... 392
--------------------------------
train...393/1296
end.... 393
--------------------------------
train...394/1296
end.... 394
--------------------------------
train...395/1296
end.... 395
--------------------------------
train...396/1296
end.... 396
--------------------------------
train...397/1296
end.... 397
--------------------------------
train...398/1296
end.... 398
--------------------------------
train...399/1296
end.... 399
--------------------------------
train...400/1296
end.... 

end.... 514
--------------------------------
train...515/1296
end.... 515
--------------------------------
train...516/1296
end.... 516
--------------------------------
train...517/1296
end.... 517
--------------------------------
train...518/1296
end.... 518
--------------------------------
train...519/1296
end.... 519
--------------------------------
train...520/1296
end.... 520
--------------------------------
train...521/1296
end.... 521
--------------------------------
train...522/1296
end.... 522
--------------------------------
train...523/1296
end.... 523
--------------------------------
train...524/1296
end.... 524
--------------------------------
train...525/1296
end.... 525
--------------------------------
train...526/1296
end.... 526
--------------------------------
train...527/1296
end.... 527
--------------------------------
train...528/1296
end.... 528
--------------------------------
train...529/1296
end.... 529
--------------------------------
train...530/1296
end.... 

end.... 647
--------------------------------
train...648/1296
end.... 648
--------------------------------
train...649/1296
end.... 649
--------------------------------
train...650/1296
end.... 650
--------------------------------
train...651/1296
end.... 651
--------------------------------
train...652/1296
end.... 652
--------------------------------
train...653/1296
end.... 653
--------------------------------
train...654/1296
end.... 654
--------------------------------
train...655/1296
end.... 655
--------------------------------
train...656/1296
end.... 656
--------------------------------
train...657/1296
end.... 657
--------------------------------
train...658/1296
end.... 658
--------------------------------
train...659/1296
end.... 659
--------------------------------
train...660/1296
end.... 660
--------------------------------
train...661/1296
end.... 661
--------------------------------
train...662/1296
end.... 662
--------------------------------
train...663/1296
end.... 

end.... 776
--------------------------------
train...777/1296
end.... 777
--------------------------------
train...778/1296
end.... 778
--------------------------------
train...779/1296
end.... 779
--------------------------------
train...780/1296
end.... 780
--------------------------------
train...781/1296
end.... 781
--------------------------------
train...782/1296
end.... 782
--------------------------------
train...783/1296
end.... 783
--------------------------------
train...784/1296
end.... 784
--------------------------------
train...785/1296
end.... 785
--------------------------------
train...786/1296
end.... 786
--------------------------------
train...787/1296
end.... 787
--------------------------------
train...788/1296
end.... 788
--------------------------------
train...789/1296
end.... 789
--------------------------------
train...790/1296
end.... 790
--------------------------------
train...791/1296
end.... 791
--------------------------------
train...792/1296
end.... 

KeyboardInterrupt: 

In [ ]:
# from sklearn.ensemble import GradientBoostingRegressor
# n = 500
# l = 0.02
# lo = "huber"
# sub = 1
# mi = 2
# ma = 11
# ms = 1

# gbrg = GradientBoostingRegressor(random_state=17,n_estimators=n,learning_rate=l,loss=lo,subsample=sub,max_depth=ma,min_samples_split=mi,min_samples_leaf=ms)
# gbrg.fit(x_train,y_train)
# pred_test = gbrg.predict(x_test)

# pred_test = pred_test.reshape(-1,1)
# sample_n = pred_test.shape[0]
# feature_n = x_test.shape[1]
# a3=calculate(y_test,pred_test,sample_n,feature_n)

# plt.figure(figsize=(20, 10))
# plt.plot(np.arange(1,len(y_test)+1),y_test, 'r-',label='Test')
# plt.plot(np.arange(1,len(pred_test)+1),pred_test, 'g-',label='pred_test')
# plt.legend(loc='best')
# plt.show()

# plt.xlabel('pred_test')
# plt.ylabel('y_test')
# plt.scatter(pred_test,y_test,marker='.')
# plt.show()